# Heart disease UCI dataset
[the dataset](https://www.kaggle.com/ronitf/heart-disease-uci)
[guide to dataset variables](https://www.kaggle.com/onatto/predicting-heart-disease-a-detailed-guide)

In [15]:
import torch
import pyro
import numpy as np
import pylab as pl
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pyro
from pyro import distributions as dist
from pyro.infer.mcmc import MCMC, HMC, NUTS
import seaborn as sns

sns.set_theme()

from scripts import *

pd.options.mode.chained_assignment = None  #hide pandas warnings

## Data exploration

In [53]:
# Import the dataset and remove id column
data = pd.read_csv("Datasets/heart_disease_UCI.csv", sep=",")


In [54]:
data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [55]:
data.columns = ['age', 'sex', 'chest_pain', 'rest_blood_pressure', 'cholesterol', 'fast_blood_sugar', 'rest_ecg', 'max_heart_rate', 'exercise_angina', 'st_depression', 'st_slope', 'major_vessels', 'thalassemia', 'target']

In [57]:
data.isnull().sum()

age                    0
sex                    0
chest_pain             0
rest_blood_pressure    0
cholesterol            0
fast_blood_sugar       0
rest_ecg               0
max_heart_rate         0
exercise_angina        0
st_depression          0
st_slope               0
major_vessels          0
thalassemia            0
target                 0
dtype: int64

In [30]:
data.dtypes

age                      int64
sex                     object
chest_pain               int64
rest_blood_pressure      int64
cholesterol              int64
fast_blood_sugar        object
rest_ecg                 int64
max_heart_rate           int64
exercise_angina          int64
st_depression          float64
st_slope                 int64
major_vessels            int64
thalassemia              int64
target                   int64
dtype: object

In [58]:
data['sex'][data['sex'] == 0] = 'female'
data['sex'][data['sex'] == 1] = 'male'
data['sex'] = data['sex'].astype('object')

data['chest_pain'][data['chest_pain'] == 0] = 'typical angina'
data['chest_pain'][data['chest_pain'] == 1] = 'atypical angina'
data['chest_pain'][data['chest_pain'] == 2] = 'non-anginal pain'
data['chest_pain'][data['chest_pain'] == 3] = 'asymptomatic'
data['chest_pain'] = data['chest_pain'].astype('object')

data['fast_blood_sugar'][data['fast_blood_sugar'] == 0] = '≤ 120mg/ml'
data['fast_blood_sugar'][data['fast_blood_sugar'] == 1] = '> 120mg/ml'
data['fast_blood_sugar'] = data['fast_blood_sugar'].astype('object')

data['rest_ecg'][data['rest_ecg'] == 0] = 'normal'
data['rest_ecg'][data['rest_ecg'] == 1] = 'ST-T wave abnormality'
data['rest_ecg'][data['rest_ecg'] == 2] = ' left ventricular hypertrophy'
data['rest_ecg'] = data['rest_ecg'].astype('object')

data['exercise_angina'][data['exercise_angina'] == 0] = 'no'
data['exercise_angina'][data['exercise_angina'] == 1] = 'yes'
data['exercise_angina'] = data['exercise_angina'].astype('object')

data['st_slope'][data['st_slope'] == 0] = 'up'
data['st_slope'][data['st_slope'] == 1] = 'flat'
data['st_slope'][data['st_slope'] == 2] = 'down'
data['st_slope'] = data['st_slope'].astype('object')

data['thalassemia'][data['thalassemia'] == 0] = 'normal'
data['thalassemia'][data['thalassemia'] == 1] = 'fixed defect'
data['thalassemia'][data['thalassemia'] == 2] = 'reversable defect'
data['thalassemia'] = data['thalassemia'].astype('object')



In [59]:
data.describe()

,age,rest_blood_pressure,cholesterol,max_heart_rate,st_depression,major_vessels,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,131.623762,246.264026,149.646865,1.039604,0.729373,0.544554
std,9.082101,17.538143,51.830751,22.905161,1.161075,1.022606,0.498835
min,29.000000,94.000000,126.000000,71.000000,0.000000,0.000000,0.000000
25%,47.500000,120.000000,211.000000,133.500000,0.000000,0.000000,0.000000
50%,55.000000,130.000000,240.000000,153.000000,0.800000,0.000000,1.000000
75%,61.000000,140.000000,274.500000,166.000000,1.600000,1.000000,1.000000
max,77.000000,200.000000,564.000000,202.000000,6.200000,4.000000,1.000000


In [60]:
data.head(10)

,age,sex,chest_pain,rest_blood_pressure,cholesterol,fast_blood_sugar,rest_ecg,max_heart_rate,exercise_angina,st_depression,st_slope,major_vessels,thalassemia,target
0,63,male,asymptomatic,145,233,> 120mg/ml,normal,150,no,2.3,up,0,fixed defect,1
1,37,male,non-anginal pain,130,250,≤ 120mg/ml,ST-T wave abnormality,187,no,3.5,up,0,reversable defect,1
2,41,female,atypical angina,130,204,≤ 120mg/ml,normal,172,no,1.4,down,0,reversable defect,1
3,56,male,atypical angina,120,236,≤ 120mg/ml,ST-T wave abnormality,178,no,0.8,down,0,reversable defect,1
4,57,female,typical angina,120,354,≤ 120mg/ml,ST-T wave abnormality,163,yes,0.6,down,0,reversable defect,1
5,57,male,typical angina,140,192,≤ 120mg/ml,ST-T wave abnormality,148,no,0.4,flat,0,fixed defect,1
6,56,female,atypical angina,140,294,≤ 120mg/ml,normal,153,no,1.3,flat,0,reversable defect,1
7,44,male,atypical angina,120,263,≤ 120mg/ml,ST-T wave abnormality,173,no,0.0,down,0,3,1
8,52,male,non-anginal pain,172,199,> 120mg/ml,ST-T wave abnormality,162,no,0.5,down,0,3,1
9,57,male,non-anginal pain,150,168,≤ 120mg/ml,ST-T wave abnormality,174,no,1.6,down,0,reversable defect,1


In [73]:
# For the categorical variables, we need to create dummies
# Drop first category of each, eg. rather than having 'male' and 'female', male' with values of 0 or 1 (1 male, and 0 female)

# useful?

data = pd.get_dummies(data, drop_first=True)
data.head(10)

,age,sex_male,rest_blood_pressure,cholesterol,max_heart_rate,st_depression,major_vessels,chest_pain_atypical angina,chest_pain_non-anginal pain,chest_pain_typical angina,fast_blood_sugar_≤ 120mg/ml,rest_ecg_ST-T wave abnormality,rest_ecg_normal,exercise_angina_yes,st_slope_flat,st_slope_up,thalassemia_fixed defect,thalassemia_normal,thalassemia_reversable defect,target
0,63,1,145,233,150,2.3,0,0,0,0,0,0,1,0,0,1,1,0,0,1
1,37,1,130,250,187,3.5,0,0,1,0,1,1,0,0,0,1,0,0,1,1
2,41,0,130,204,172,1.4,0,1,0,0,1,0,1,0,0,0,0,0,1,1
3,56,1,120,236,178,0.8,0,1,0,0,1,1,0,0,0,0,0,0,1,1
4,57,0,120,354,163,0.6,0,0,0,1,1,1,0,1,0,0,0,0,1,1
5,57,1,140,192,148,0.4,0,0,0,1,1,1,0,0,1,0,1,0,0,1
6,56,0,140,294,153,1.3,0,1,0,0,1,0,1,0,1,0,0,0,1,1
7,44,1,120,263,173,0.0,0,1,0,0,1,1,0,0,0,0,0,0,0,1
8,52,1,172,199,162,0.5,0,0,1,0,0,1,0,0,0,0,0,0,0,1
9,57,1,150,168,174,1.6,0,0,1,0,1,1,0,0,0,0,0,0,1,1


In [72]:
# reorder columns to have target as last and sex as second

data = data[['age', 'sex_male', 'rest_blood_pressure', 'cholesterol', 'max_heart_rate', 'st_depression', 'major_vessels', 'chest_pain_atypical angina', 'chest_pain_non-anginal pain', 'chest_pain_typical angina', 'fast_blood_sugar_≤ 120mg/ml', 'rest_ecg_ST-T wave abnormality', 'rest_ecg_normal', 'exercise_angina_yes', 'st_slope_flat', 'st_slope_up', 'thalassemia_fixed defect', 'thalassemia_normal', 'thalassemia_reversable defect',  'target']]

## Data analysis

In [74]:
# Split into train and test set 
data = dataframe_to_tensor(data)

X, y = data[:, :-1], data[:, -1]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.8)